## Imports

In [1]:
import os
from dotenv import load_dotenv

import random
from typing import List, Dict

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.callbacks import get_openai_callback
import tiktoken

## Utility Functions

In [2]:
def preview_formatted_prompt(question: str, retriever, prompt_template):
    """
    Recovers the relevant context for the passed `question`, based on the passed `retriever`, 
    and combines these as a formatted prompt, based on the `prompt_template`.
    """
    pages = retriever.invoke(question)
    
    context = "\n\n".join([page.page_content for page in pages])
    
    formatted_prompt = prompt_template.format(context=context, question=question)
    
    return formatted_prompt

def get_random_examples(examples: List[Dict[str, str]], n: int = 1) -> List[Dict[str, str]]:
    """
    Returns a `n` random examples from the passed list of examples.
    """
    return random.sample(examples, n)

def count_tokens(text: str, model_name: str) -> int:
    """
    Counts the number of tokens in the passed `text` using the specified model's encoding.
    """
    enc = tiktoken.encoding_for_model(model_name)
    tokens = enc.encode(text)
    return len(tokens)

def ask_llm(question: str, llm_chain, retriever, verbose: bool = True) -> tuple:
    """
    Recovers the relevant context for the passed `question`, based on the passed `retriever`, 
    and invokes the LLM that is part of the `llm_chain` to recieve an answer. The context, as well
    as a question and answer pair, is returned.
    """
    with get_openai_callback() as cb:
        pages = retriever.invoke(question)
        
        context = "\n\n".join([page.page_content for page in pages])
        
        question_and_answer = llm_chain.invoke(question)

        if verbose:
            print(f"Total Tokens: {cb.total_tokens}")
            print(f"Prompt Tokens: {cb.prompt_tokens}")
            print(f"Completion Tokens: {cb.completion_tokens}")
            print(f"Total Cost (USD): ${cb.total_cost}")
    
    return context, question_and_answer

## LLM and RAG Setup

Before answering questions using a LLM (either open or closed source) we need to load the respective LLM, as well as the documents to be used as part of RAG. To begine with, we will use the `gpt-3.5-turbo-1106` model, and then later the `gpt-4-turbo` model.

**To be clear, we are using only OpenAI models in this notebook.**

In [3]:
# Load environment variables from `.env` file securely
load_dotenv()

# Set OpenAI API key for API access
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [4]:
# Create loader to read and split `.pdf` documents
loader = PyMuPDFLoader('How ASML took over the chipmaking chessboard (MIT 010424).pdf')

# Load and split `.pdf` document by pages
pages = loader.load_and_split()

In [5]:
# Quick check to see that `.pdf` is laoded as expected
for page in pages:
    print(page)

page_content='1 
 
How ASML took over the chipmaking chessboard 
MIT Technology Review sat down with outgoing CTO Martin van den Brink to talk about the company’s rise to 
dominance and the life and death of Moore’s Law. 
 
By Mat Honanarchive pageJames O'Donnellarchive page 
April 1, 2024 
 
On a drab Monday morning in San Jose, California, at the drab San Jose Convention Center, attendees of the SPIE 
Advanced Lithography and Patterning Conference filed into the main ballroom until all the seats were taken and the 
crowd began to line the walls along the back and sides of the room. The convention brings together people who work in 
the chip industry from all over the world. And on this cool February morning, they had gathered to hear tech industry 
luminaries extol the late Gordon Moore, Intel’s cofounder and first CEO.  
 
Craig Barrett, also a former CEO of Intel, paid tribute, as did the legendary engineer Burn-Jeng Lin, a pioneer of 
immersion lithography, a patterning technology

In [6]:
# Utilise OpenAI vector embeddings
embeddings = OpenAIEmbeddings()

# Store embeddings, for each page, using ChromaDB
stored_embeddings = Chroma.from_documents(pages, embeddings, collection_name='asml_report')

# Create a naive retriever (cosine similarity for each page)
retriever = stored_embeddings.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# Maximum marginal relevance retriever
retriever_mmr = stored_embeddings.as_retriever(search_type="mmr", search_kwargs={"k": 3})

The Maximum Marginal Relevance (MMR) algorithm is designed to optimize the balance between relevance and diversity in information retrieval and summarization tasks. Here is a concise overview:


- Define relevance as the match quality between a document and a query, and diversity as the degree to which documents differ from each other.
- Start with a query and an initial set of document candidates.
- **MMR Score Calculation**

    * Calculate relevance scores using similarity metrics like cosine similarity or TF-IDF.
    * Compute similarity scores between each candidate and selected documents to assess diversity.
    * Derive the MMR score as: $\text{MMR} = \lambda \times \text{Relevance} - (1 - \lambda) \times \text{Max Similarity}$
    where $\lambda$ balances relevance (higher values) and diversity (lower values).

-  Choose the document with the highest MMR score, add it to the result set, and remove it from candidates.
-  Repeat scoring and selection until the desired number of documents is reached or other stopping criteria are met.
-  The result is a set of documents that are relevant to the query and diverse across different aspects.

This method is particularly useful in scenarios where exposure to varied perspectives is beneficial.


### Pricing

![title](pricing.png)

For a full list of OpenAI API pricing, see: https://openai.com/api/pricing/.

In [7]:
# Define (OpenAI) model hyperparameters to use (See pricing above!)
LLM_HYPERPARAMETERS = {
    'MODEL_NAME': 'gpt-3.5-turbo-1106',
    'CHAT_MODEL': True,
    'TEMPERATURE': 0,
    'VERBOSE': True,
    'MAX_OUTPUT_TOKENS': -1 # Set to -1 for no limit
}

# Instantiate (Chat) OpenAI LLM
if LLM_HYPERPARAMETERS['CHAT_MODEL'] == True:
    llm = ChatOpenAI(
        model=LLM_HYPERPARAMETERS['MODEL_NAME'], 
        temperature=LLM_HYPERPARAMETERS['TEMPERATURE'], 
        verbose=LLM_HYPERPARAMETERS['VERBOSE']
    )
else:
    llm = OpenAI(
        model=LLM_HYPERPARAMETERS['MODEL_NAME'], 
        temperature=LLM_HYPERPARAMETERS['TEMPERATURE'], 
        verbose=LLM_HYPERPARAMETERS['VERBOSE']
    )

In [9]:
# Create prompt template
template = """You are a world-leading financial analyst. Please provide a comprehensive and illuminating answer to the following question based on the given context. DO NOT BE LAZY!

Context:
{context}

Q: {question}

A:"""

zero_shot_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

# Create RetrievalQA chain
qa_chain_zero_shot = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": zero_shot_prompt_template},
    verbose=True
)

In [10]:
print(qa_chain_zero_shot.combine_documents_chain.llm_chain.prompt.template)

You are a world-leading financial analyst. Please provide a comprehensive and illuminating answer to the following question based on the given context. DO NOT BE LAZY!

Context:
{context}

Q: {question}

A:


In [11]:
print(preview_formatted_prompt("MY_QUESTION", retriever, zero_shot_prompt_template))

You are a world-leading financial analyst. Please provide a comprehensive and illuminating answer to the following question based on the given context. DO NOT BE LAZY!

Context:
2 
 
 
Betting big on an unwieldy wavelength 
 
ASML is such an undisputed leader in today’s chip ecosystem that it’s hard to believe the company’s market dominance 
really only dates back to 2017, when its EUV machine, after 17 years of development, upended the conventional process 
for making chips.  
 
Since the 1960s, photolithography has made it possible to pack computer chips with more and more components. The 
process involves crafting small circuits by guiding beams of light through a series of mirrors and lenses and then shining 
that light on a mask, which contains a pattern. Light conveys the chip design, layer by layer, eventually building circuits 
that form the computational building blocks of everything from smartphones to artificial intelligence. 
 
Photolithographers have a limited set of tools

In [12]:
examples = [
    {"question": "Who is the outgoing CTO of ASML?", "answer": "The outgoing CTO of ASML is Van den Brink."},
]

# Create prompt template for each example
example_template = """
Question: {question}
Answer: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template=example_template
)

# Create few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=get_random_examples(examples, n=1),
    example_prompt=example_prompt,
    prefix="""You are a world-leading financial analyst. Please provide a comprehensive and illuminating answer to the following question based on the given context and examples. DO NOT BE LAZY!

Here are some examples:""",
    suffix="""
Context:
{context}

Question: {question}

Answer:""",
    input_variables=["context", "question"],
    example_separator="\n\n"
)

# Create RetrievalQA chain
qa_chain_few_shot = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": few_shot_prompt_template},
    verbose=True
)

In [13]:
print(preview_formatted_prompt("MY_QUESTION", retriever, few_shot_prompt_template))

You are a world-leading financial analyst. Please provide a comprehensive and illuminating answer to the following question based on the given context and examples. DO NOT BE LAZY!

Here are some examples:


Question: Who is the outgoing CTO of ASML?
Answer: The outgoing CTO of ASML is Van den Brink.



Context:
2 
 
 
Betting big on an unwieldy wavelength 
 
ASML is such an undisputed leader in today’s chip ecosystem that it’s hard to believe the company’s market dominance 
really only dates back to 2017, when its EUV machine, after 17 years of development, upended the conventional process 
for making chips.  
 
Since the 1960s, photolithography has made it possible to pack computer chips with more and more components. The 
process involves crafting small circuits by guiding beams of light through a series of mirrors and lenses and then shining 
that light on a mask, which contains a pattern. Light conveys the chip design, layer by layer, eventually building circuits 
that form the co

In [19]:
question_1 = "Generate a concise summary (1-2 paragraphs) highlighting the key points of the document."
context_q1, result_q1 = ask_llm(question_1, qa_chain_zero_shot, retriever)



> Entering new RetrievalQA chain...

> Finished chain.
Total Tokens: 2076
Prompt Tokens: 1906
Completion Tokens: 170
Total Cost (USD): $0.0022459999999999997


In [18]:
# qa_chain_zero_shot_mmr = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever_mmr,
#     chain_type_kwargs={"prompt": zero_shot_prompt_template},
#     verbose=True
# )

# question_1 = "Generate a concise summary (1-2 paragraphs) highlighting the key points of the document."
# context_q1, result_q1 = ask_llm(question_1, qa_chain_zero_shot_mmr, retriever_mmr)

In [20]:
num_tokens = count_tokens(result_q1['result'], LLM_HYPERPARAMETERS['MODEL_NAME'])
print(f"Number of tokens in the output: {num_tokens}")

Number of tokens in the output: 170


In [21]:
print(result_q1['query'])

Generate a concise summary (1-2 paragraphs) highlighting the key points of the document.


In [22]:
print(context_q1)

smoothness of mirrors destined for ASML. The aim was to reduce the number of costly repercussions the change would 
have on the rest of the supply chain, like the companies that make reticles containing the designs of the chips.

2 
 
 
Betting big on an unwieldy wavelength 
 
ASML is such an undisputed leader in today’s chip ecosystem that it’s hard to believe the company’s market dominance 
really only dates back to 2017, when its EUV machine, after 17 years of development, upended the conventional process 
for making chips.  
 
Since the 1960s, photolithography has made it possible to pack computer chips with more and more components. The 
process involves crafting small circuits by guiding beams of light through a series of mirrors and lenses and then shining 
that light on a mask, which contains a pattern. Light conveys the chip design, layer by layer, eventually building circuits 
that form the computational building blocks of everything from smartphones to artificial intelligenc

In [23]:
print(result_q1['result'])

ASML, a Dutch company, has become a dominant force in the chip industry since the development of its EUV machine in 2017. This machine revolutionized the process of making chips by using extreme-ultraviolet (EUV) lithography, which allows for the creation of smaller components on chips. ASML's technology has enabled the industry to maintain Moore's Law, which dictates that the number of transistors on an integrated circuit doubles every two years. The company's success has been attributed to its investment in creating tools capable of making ever finer features, such as the EUV lithography machines. ASML's ability to meet the demands of Moore's Law has earned it widespread recognition and a standing ovation at a tech industry conference. However, the company faces ongoing pressure to continue staying ahead of the demands of Moore's Law.


## Question 2

In [24]:
question_2 = "Create a bullet-point summary containing 5 main ideas from the document."
context_q2, result_q2 = ask_llm(question_2, qa_chain_zero_shot, retriever)



> Entering new RetrievalQA chain...

> Finished chain.
Total Tokens: 2078
Prompt Tokens: 1902
Completion Tokens: 176
Total Cost (USD): $0.002254


In [25]:
print(result_q2['result'])

- ASML, a Dutch company, has become the dominant force in the chip industry, particularly in the production of advanced computer chips.
- The company's machines have been crucial in maintaining Moore's Law, which dictates the doubling of transistors on an integrated circuit every two years.
- ASML's development of extreme-ultraviolet (EUV) lithography machines has been a game-changer in the chipmaking process, allowing for the creation of smaller components on chips.
- The company faced significant challenges in developing EUV technology, including the need to create, capture, and focus extremely small wavelengths of light and to build a supply chain of specialized materials, such as smooth mirrors.
- ASML's success in staying ahead of the demands of Moore's Law has solidified its market dominance, with competitors like Nikon and Canon dropping out of the pursuit of EUV technology.


## Question 3

In [26]:
question_3a = "What are the drivers of ASML's growth?"
context_q3a, result_q3a = ask_llm(question_3a, qa_chain_zero_shot, retriever)



> Entering new RetrievalQA chain...

> Finished chain.
Total Tokens: 2366
Prompt Tokens: 1894
Completion Tokens: 472
Total Cost (USD): $0.002838


In [27]:
print(result_q3a['result'])

ASML's growth can be attributed to several key drivers. Firstly, the company's investment in research and development (R&D) has been a major factor in its success. ASML has consistently invested in creating tools capable of making ever finer features, such as the extreme-ultraviolet (EUV) lithography machines it rolled out in 2017, the high-numerical-aperture (high-NA) EUV machines it is currently rolling out, and the hyper-NA EUV machines it has planned for the future. This commitment to innovation has allowed ASML to stay ahead of the demands of Moore's Law and maintain its position as a leader in the industry.

Additionally, ASML's control of the supply chain has played a significant role in its growth. The company has strategically acquired companies it relies on, such as Cymer, a maker of light sources, allowing it to control key components of its manufacturing process. This control of the supply chain has extended to ASML's customers as well, with the company offering shares to i

In [28]:
question_3b = "What is the next big idea for ASML?"
context_q3b, result_q3b = ask_llm(question_3b, qa_chain_zero_shot, retriever)



> Entering new RetrievalQA chain...

> Finished chain.
Total Tokens: 2202
Prompt Tokens: 1936
Completion Tokens: 266
Total Cost (USD): $0.002468


In [29]:
print(result_q3b['result'])

The next big idea for ASML is to continue pushing the boundaries of chipmaking technology by developing and implementing even more advanced lithography machines. Specifically, ASML is focused on further refining and improving extreme-ultraviolet (EUV) lithography, as well as developing high-numerical-aperture (high-NA) EUV machines and potentially hyper-NA EUV machines in the future. These machines are crucial for enabling chipmakers to continue shrinking transistor dimensions and packing more transistors onto microchips, in line with the principles of Moore's Law.

ASML's investment in creating tools capable of making ever finer features, such as the EUV lithography machines, demonstrates the company's commitment to staying at the forefront of technological innovation in the chip industry. By continuously improving and advancing lithography technology, ASML aims to ensure that progress in chip manufacturing remains possible, despite the increasing demands of Moore's Law.

In summary, 

## Question 4

Here we utilise few-shot prompting for the critique of the thesis!

This can viewed as in-context learning (that of course is not permanent). Can be viewed somewhat analgously to fine-tuning in that we aim to shift the distribution of predicted tokens based on domain/task-specific data.

In [9]:
# Create prompt template
thesis_template = """You are a world-leading financial analyst. Please provide a simple investment thesis for the given company.

Context:
{context}

Q: {question}

A:"""

zero_shot_thesis_template = PromptTemplate(
    input_variables=["context", "question"],
    template=thesis_template
)

# Create RetrievalQA chain
thesis_chain_zero_shot = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": zero_shot_thesis_template},
    verbose=True
)

company = "ASML"
company_and_thesis_original = thesis_chain_zero_shot.invoke(f"Generate an investment thesis for {company}")
original_thesis = company_and_thesis_original['result']

print("Original Investment Thesis:")
print(original_thesis)



> Entering new RetrievalQA chain...

> Finished chain.
Original Investment Thesis:
ASML is a dominant player in the semiconductor manufacturing industry, with a history of successful innovation and technological advancement. The company's EUV machines have revolutionized the chip-making process, leading to increased demand from major semiconductor manufacturers. ASML's control of the supply chain and its ability to stay ahead of technological advancements position it as a strong investment opportunity. Despite initial challenges, the company has demonstrated its ability to overcome obstacles and maintain market dominance. With a track record of successful R&D investments and a monopoly position in the industry, ASML presents a compelling investment opportunity for long-term growth and potential returns.


In [28]:
critique_examples = [
    {
        "question": "Investment thesis for ASML: ASML is poised for long-term growth due to its monopoly in EUV lithography machines, essential for advanced semiconductor manufacturing. With increasing demand for high-performance chips in AI, 5G, and IoT applications, ASML's technology is critical. Their strong R&D investments and partnerships with major chip manufacturers ensure their continued market leadership.",
        "answer": "Strengths: Recognizes ASML's monopoly in EUV lithography, growing demand for advanced chips, and strong R&D investments. Weaknesses: Lacks specific financial metrics, doesn't address potential risks like geopolitical tensions or cyclical nature of semiconductor industry. Could be improved by including quantitative growth projections and discussing ASML's valuation relative to peers."
    }
]

critique_example_template = """
Thesis: {question}
Critique: {answer}
"""

critique_example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template=critique_example_template
)

critique_prompt_template = FewShotPromptTemplate(
    examples=get_random_examples(critique_examples, 1),
    example_prompt=critique_example_prompt,
    prefix="""You are a world-leading, and sceptical financial analyst. Please provide a critique of the given investment thesis, highlighting its strengths, weaknesses, and references. PLEASE PROVIDE EXPLICIT REFERENCES, WITH QUOTES, FROM THE PROVIDED CONTEXT WHERE APPROPRIATE! If there are references that contradict the provided thesis, please provide these.
Here is an example:""",
    suffix="""
Context:
{context}

Thesis: {question}
Critique:""",
    input_variables=["context", "question"],
    example_separator="\n\n"
)

critique_chain_few_shot = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": critique_prompt_template},
    verbose=True
)

In [29]:
thesis_and_critique = critique_chain_few_shot.invoke(original_thesis)



> Entering new RetrievalQA chain...

> Finished chain.


In [30]:
print(thesis_and_critique['result'])

Strengths: The thesis accurately highlights ASML's historical success in innovation and technological advancement, as well as its monopoly position in the industry. It also acknowledges the company's control of the supply chain and its ability to stay ahead of technological advancements. The mention of successful R&D investments and the potential for long-term growth and returns also adds credibility to the thesis.

Weaknesses: The thesis lacks specific financial metrics to support the investment opportunity, such as revenue growth, profit margins, or valuation metrics. It also does not address potential risks, such as geopolitical tensions or the cyclical nature of the semiconductor industry, which could impact ASML's future performance. Additionally, the thesis could be improved by including quantitative growth projections and discussing ASML's valuation relative to peers.

References:
- The thesis accurately highlights ASML's historical success in innovation and technological advanc

What about an incredibly basic investment thesis, with no substance whatsoever?

In [31]:
bad_thesis = "ASML is a rubbish company and you should not buy it!"
bad_thesis_and_critique = critique_chain_few_shot.invoke(bad_thesis)



> Entering new RetrievalQA chain...

> Finished chain.


In [32]:
print(bad_thesis_and_critique['result'])

The provided context does not support the thesis that ASML is a rubbish company and should not be bought. In fact, the context provides evidence of ASML's market dominance and technological advancements in the semiconductor industry. The company's monopoly in EUV lithography machines, essential for advanced semiconductor manufacturing, is highlighted, as well as its strong R&D investments and partnerships with major chip manufacturers. The context also emphasizes ASML's role in maintaining Moore's Law and meeting the increasing demands for advanced computer chips.

Strengths: The context supports the strengths of ASML, including its market dominance in EUV lithography, technological advancements, and ability to meet the demands of Moore's Law.

Weaknesses: The thesis lacks specific evidence or references to support the claim that ASML is a rubbish company and should not be bought. It does not address the company's financial performance, market position, or any specific weaknesses that 

#### Now using gpt4 model...

In [34]:
# Define (OpenAI) model hyperparameters to use (See pricing above!)
LLM_HYPERPARAMETERS = {
    'MODEL_NAME': 'gpt-4-turbo',
    'CHAT_MODEL': True,
    'TEMPERATURE': 0,
    'VERBOSE': True,
    'MAX_OUTPUT_TOKENS': -1 # Set to -1 for no limit
}

# Instantiate (Chat) OpenAI LLM
if LLM_HYPERPARAMETERS['CHAT_MODEL'] == True:
    llm_gpt4 = ChatOpenAI(
        model=LLM_HYPERPARAMETERS['MODEL_NAME'], 
        temperature=LLM_HYPERPARAMETERS['TEMPERATURE'], 
        verbose=LLM_HYPERPARAMETERS['VERBOSE']
    )
else:
    llm_gpt4 = OpenAI(
        model=LLM_HYPERPARAMETERS['MODEL_NAME'], 
        temperature=LLM_HYPERPARAMETERS['TEMPERATURE'], 
        verbose=LLM_HYPERPARAMETERS['VERBOSE']
    )

In [35]:
thesis_chain_zero_shot_gpt4 = RetrievalQA.from_chain_type(
    llm=llm_gpt4,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": zero_shot_thesis_template},
    verbose=True
)

company = "ASML"
company_and_thesis_original = thesis_chain_zero_shot_gpt4.invoke(f"Generate an investment thesis for {company}")
original_thesis = company_and_thesis_original['result']

print("Original Investment Thesis:")
print(original_thesis)



> Entering new RetrievalQA chain...

> Finished chain.
Original Investment Thesis:
**Investment Thesis for ASML**

**1. Market Leadership and Technological Edge:**
ASML has established itself as a dominant player in the semiconductor equipment industry, particularly in the field of extreme-ultraviolet (EUV) lithography. Since successfully commercializing EUV technology, ASML has effectively secured a monopoly in this high-precision area, with its machines being critical for leading chipmakers like Intel, Samsung, and TSMC. This technological edge provides ASML with a significant competitive moat, as alternative technologies and competitors have not reached similar levels of development or adoption.

**2. Critical Role in Advancing Moore's Law:**
ASML's technology is pivotal in enabling the continuation of Moore's Law, which posits the doubling of transistors on a microchip approximately every two years, thereby increasing performance. ASML’s advancements in lithography, including its

In [37]:
critique_chain_few_shot_gpt4 = RetrievalQA.from_chain_type(
    llm=llm_gpt4,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": critique_prompt_template},
    verbose=True
)

thesis_and_critique = critique_chain_few_shot_gpt4.invoke(original_thesis)



> Entering new RetrievalQA chain...

> Finished chain.


In [38]:
print(thesis_and_critique['result'])

**Critique of the Investment Thesis for ASML**

**Strengths:**
1. **Market Leadership and Technological Edge:** The thesis accurately highlights ASML's dominance in the EUV lithography sector, a critical technology for advanced semiconductor manufacturing. This is supported by the context stating, "ASML is such an undisputed leader in today’s chip ecosystem" and "ASML’s bet began to pay off" with the successful shipment and adoption of EUV machines by major chipmakers (Context 3).

2. **Critical Role in Advancing Moore's Law:** The thesis correctly identifies ASML's pivotal role in sustaining Moore's Law through technological advancements in EUV lithography. This is echoed by van den Brink's statement during the SPIE Advanced Lithography and Patterning Conference, emphasizing ASML's contribution to maintaining Moore's Law (Context 1).

3. **Strong Financial Performance and Customer Base:** The thesis points out ASML's robust financial growth and strategic customer relationships, which 

In [39]:
bad_thesis = "ASML is a rubbish company and you should not buy it!"
bad_thesis_and_critique = critique_chain_few_shot_gpt4.invoke(bad_thesis)




> Entering new RetrievalQA chain...

> Finished chain.


In [40]:
print(bad_thesis_and_critique['result'])

Critique of the Thesis: "ASML is a rubbish company and you should not buy it!"

**Strengths:**
- The thesis is clear and direct in its stance against investing in ASML.

**Weaknesses:**
1. **Lack of Evidence and Contextual Support:** The thesis provides no evidence or detailed reasoning to support the claim that ASML is a "rubbish company." This is a significant oversight, especially given the extensive context provided which highlights ASML's pivotal role and technological advancements in the semiconductor industry.
   
2. **Contradiction with Provided Context:**
   - The context explicitly states ASML's critical role in maintaining Moore's Law through its advanced EUV lithography machines. Martin van den Brink, the outgoing CTO, emphasized at the SPIE Advanced Lithography and Patterning Conference that ASML's technology is essential for continuing to meet the demands of modern computing, including AI ("Today, they are the only ones in the world capable of producing circuitry at the d